In [239]:
import pandas as pd
import re
from tqdm import tqdm

In [219]:
df = pd.read_csv('data/processed_data.csv')
df.tail()

,label,subject,email_to,email_from,message
75414,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75415,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75416,0,"Re: [R] Me again, about the horrible documenta...",Duncan Murdoch <murdoch@stats.uwo.ca>,Philippe Grosjean <phgrosjean@sciviews.org>,"For those who are interested, I just cook a li..."
75417,0,Re: [R] RODBC problem,<r-help@stat.math.ethz.ch>,=?iso-8859-1?Q?Bernhard_Wellh=F6fer?=\n\t<Bern...,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan..."
75418,1,I wanted the desk at his own laws: of the. Bu...,the00@plg.uwaterloo.ca,"""Danny"" <pwcusnt@noblecoffee.com>",Content-Type: multipart/alternative;\n\tbounda...


In [153]:
text = df.loc[75385]['message']
print(text)

> Package: dcraw
> Version: 7.02-1
> Severity: serious
>
> On Thu, 05 Jul 2007, Steve King wrote:
>> However I would appreciate it if the assembled masses of legal
>> experts could confirm that they agree that this is the case.
>
> There's actually an even more fundamental problem with dcraw.c:
>
> /*
>    dcraw.c -- Dave Coffin's raw photo decoder
>    Copyright 1997-2007 by Dave Coffin, dcoffin a cybercom o net
>
>    This is a command-line ANSI C program to convert raw photos from
>    any digital camera on any computer running any operating system.
>
>    No license is required to download and use dcraw.c.  However,
>    to lawfully redistribute this code, you must either (a) include
>    full source code* for all executable files containing RESTRICTED
>    functions, (b) remove all RESTRICTED functions, re-implement them,
>    or copy them from an earlier, unrestricted Revision of dcraw.c,
>    or (c) purchase a license from the author.
>
>    The functions that process Foveon ima

In [160]:
def clean_email(email):
    expressions = re.sub(r'Content-Type: \S+|Content-Transfer-Encoding: \S+|charset=\S+|tcharset=\S+|On\s.*?wrote:|0x[0-9]+|boundary=\S+|\S+NextPart\S+', '', email, flags=re.IGNORECASE)
    links_emails = re.sub(r'\S+@\S+|https?://\S+|www\.\S+', '', expressions, flags=re.IGNORECASE)
    other_characters = re.sub(r'[^a-zA-Z_ ]|_+', " ", links_emails, flags=re.IGNORECASE)
    multi_spaces = re.sub(r' {2,}'," ", other_characters)
    return multi_spaces

In [161]:
clean_email(df.loc[75385]['message'])

' Package dcraw Version Severity serious However I would appreciate it if the assembled masses of legal experts could confirm that they agree that this is the case There s actually an even more fundamental problem with dcraw c dcraw c Dave Coffin s raw photo decoder Copyright by Dave Coffin dcoffin a cybercom o net This is a command line ANSI C program to convert raw photos from any digital camera on any computer running any operating system No license is required to download and use dcraw c However to lawfully redistribute this code you must either a include full source code for all executable files containing RESTRICTED functions b remove all RESTRICTED functions re implement them or copy them from an earlier unrestricted Revision of dcraw c or c purchase a license from the author The functions that process Foveon images have been RESTRICTED since Revision All other code remains free for all uses If you have not modified dcraw c in any way a link to my homepage qualifies as full sour

In [171]:
messages = df['message'].astype(str)
cleaned_messages = messages.apply(clean_email)

In [220]:
df_cleaned = pd.DataFrame()
df_cleaned['label'] = df['label']
df_cleaned['subject'] = df['subject']
df_cleaned['email_to'] = df['email_from']
df_cleaned['message'] = cleaned_messages
df_cleaned.to_parquet('data/cleaned_messages.parquet')

In [221]:
df_cleaned

,label,subject,email_to,message
0,1,"Generic Cialis, branded quality@","""Tomas Jacobs"" <RickyAmes@aol.com>",Do you feel the pressure to perform and not r...
1,0,Typo in /debian/README,Yan Morin <yan.morin@savoirfairelinux.com>,Hi i ve just updated from the gulus and I chec...
2,1,authentic viagra,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...",Mega authenticV I A G R A DISCOUNT priceC I A...
3,1,Nice talking with ya,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>",Hey Billy it was really fun going out the othe...
4,1,or trembling; stomach cramps; trouble in sleep...,"""Christi T. Jernigan"" <dcube@totalink.net>",system of the home It will have the capabilit...
...,...,...,...,...
75414,1,the reply for your request for a job place [le...,"""Sydney Car Centre"" <Merrill8783@168city.com>",While we may have high expectations of our as...
75415,1,the reply for your request for a job place [le...,"""Sydney Car Centre"" <Merrill8783@168city.com>",While we may have high expectations of our as...
75416,0,"Re: [R] Me again, about the horrible documenta...",Philippe Grosjean <phgrosjean@sciviews.org>,For those who are interested I just cook a lit...
75417,0,Re: [R] RODBC problem,=?iso-8859-1?Q?Bernhard_Wellh=F6fer?=\n\t<Bern...,Hello as I wrote I call sqlFetch channel t stu...


In [178]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

/home/michal/Software/anaconda3/envs/torch2/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [179]:
def tokenize_message(cleaned_message):
    token = tokenizer(cleaned_message, return_tensors='pt', padding=True, truncation=True, max_length=512)
    return token

In [184]:
tokens = []
for message in df_cleaned['message']:
    token = tokenize_message(message)
    tokens.append(token)

In [215]:
torch.save(tokens, 'data/messages_tokens.pt')

In [236]:
def get_bert_embeddings(token):
    with torch.no_grad():
        embedding = model(token['input_ids'], attention_mask=token['attention_mask'])
        return embedding.pooler_output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

progress_bar = tqdm(total=len(tokens), desc='Processing')
embeddings = []
for token in tokens:
    token.to(device)
    embedding = get_bert_embeddings(token)
    embeddings.append(embedding)
    progress_bar.update(1)
    del token
progress_bar.close()

torch.save(embeddings, 'data/embeddings.pt')

In [271]:
embs = torch.load('data/embeddings.pt')

In [283]:
embeddings_no_tensor = []
for emb in embs:
    embedding = emb[0].tolist()
    embeddings_no_tensor.append(embedding)

In [286]:
df_embeddings = pd.DataFrame()
df_embeddings[['label', 'subject', 'email_to', 'email_from']] = df[['label', 'subject', 'email_to', 'email_from']]
df_embeddings['embedding'] = embeddings_no_tensor
df_embeddings.to_parquet('data/bert_embeddings.parquet')

In [287]:
df_embeddings

,label,subject,email_to,email_from,embedding
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>","[-0.8508955836296082, -0.3548078238964081, -0...."
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"[-0.7363540530204773, -0.4528566002845764, -0...."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","[-0.8038183450698853, -0.551634669303894, -0.9..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","[-0.8466466665267944, -0.536900520324707, -0.8..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>","[-0.5881412029266357, -0.4939061105251312, -0...."
...,...,...,...,...,...
75414,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>","[-0.5784388184547424, -0.4496438503265381, -0...."
75415,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>","[-0.5784388184547424, -0.4496438503265381, -0...."
75416,0,"Re: [R] Me again, about the horrible documenta...",Duncan Murdoch <murdoch@stats.uwo.ca>,Philippe Grosjean <phgrosjean@sciviews.org>,"[-0.5956807136535645, -0.6195296049118042, -0...."
75417,0,Re: [R] RODBC problem,<r-help@stat.math.ethz.ch>,=?iso-8859-1?Q?Bernhard_Wellh=F6fer?=\n\t<Bern...,"[-0.7729495167732239, -0.7134729623794556, -0...."
